# HDFC Data Analysis - Enhanced CEO Dashboard

This notebook demonstrates enhanced visualization capabilities for CEO presentations, allowing full customization of:
- Chart titles and subtitles
- Axis labels
- Color schemes
- Visual styles
- Value formatting

The visualizations are designed to be visually appealing and professional for executive-level presentations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from hdfc_viz import plot_bar_chart, create_dashboard, COLOR_SCHEMES

# Set the plotting style
plt.style.use('ggplot')
sns.set_context("talk")

# Load the data
df = pd.read_excel('HDFC_modified.xlsx', index_col="Category", sheet_name="WorkStatus")
df.head()

## Basic Customization Example

This example shows how to create a simple bar chart with custom titles, colors, and formatting.

In [ ]:
# Select first two columns to plot
first_two_cols = df.iloc[:, :2]

# Plot with custom titles and colors
fig, axes = plot_bar_chart(
    df=first_two_cols,
    columns=first_two_cols.columns,
    title="Cohort-wise Head Count Distribution",
    subtitle="Analysis of CAP LRM and CAP 12 Cohorts",
    column_titles={
        "CAP LRM cohort": "LRM Cohort\nHead Count",
        "CAP 12  cohort": "Cohort 12\nHead Count"
    },
    xlabel="Work Status",
    ylabels="Number of Employees",
    color_scheme='hdfc_brand',
    bg_style='presentation',
    bar_edge_color='white',
    bar_edge_width=0.5,
    bar_alpha=0.85
)

## Advanced Formatting with Percentages

This example demonstrates proper formatting of percentage values and enhanced visual style.

In [ ]:
# Select percentage columns
percentage_cols = [col for col in df.columns if '%' in col][:2]

# Create custom formatting instructions
value_format = {
    col: {'is_percentage': True, 'precision': 1} for col in percentage_cols
}

# Plot with percentage formatting
fig, axes = plot_bar_chart(
    df=df,
    columns=percentage_cols,
    title="Performance Percentage Analysis",
    subtitle="Percentage Metrics for Active vs Inactive Status",
    color_scheme='gradient_blue',
    value_format=value_format,
    bar_edge_color='gray',
    bar_edge_width=0.3,
    bar_width=0.6,
    show_values=True,
    value_fontsize=12,
    grid=True
)

## Full Dashboard with Multiple Groups

This example shows how to create a complete dashboard with multiple groups of visualizations.

In [ ]:
# Define column groups for the dashboard
column_groups = [
    df.columns[:2].tolist(),           # First 2 columns
    df.columns[2:6].tolist(),          # Columns 3-6
    df.columns[6:10].tolist(),         # Columns 7-10
    df.columns[10:12].tolist(),        # Columns 11-12
    df.columns[12:].tolist()           # Columns 13+
]

# Define custom titles for each group
custom_titles = {
    0: "Cohort Head Count Analysis",
    1: "Performance Achievement Analysis",
    2: "KPI Performance Metrics",
    3: "Time to Sale and CAR2CATPO Analysis",
    4: "Attrition and Residency Analysis"
}

# Define custom subtitles
custom_subtitles = {
    0: "Distribution of headcount by work status across cohorts",
    1: "Comparison of KPI performance achievement percentages",
    2: "Performance multiples and KPI metrics analysis",
    3: "Time to first sale and conversion ratios",
    4: "Employee retention and attrition indicators"
}

# Define value formatting by column type
value_formats = {}
for i, group in enumerate(column_groups):
    value_formats[i] = {}
    for col in group:
        if '%' in col.lower():
            value_formats[i][col] = {'is_percentage': True, 'precision': 1}
        elif 'ratio' in col.lower():
            value_formats[i][col] = {'is_percentage': False, 'precision': 2}
        elif 'time' in col.lower() or 'month' in col.lower():
            value_formats[i][col] = {'is_percentage': False, 'precision': 1}
        else:
            value_formats[i][col] = {'is_percentage': False, 'precision': 0}

# Create customized column titles for better readability
custom_column_titles = {}
for i, group in enumerate(column_groups):
    custom_column_titles[i] = {}
    for col in group:
        words = col.split()
        # Create a more concise title
        if len(words) > 5:
            # Break long titles into multiple lines
            if 'CAP' in col and 'cohort' in col.lower():
                custom_column_titles[i][col] = col.replace('CAP', '\nCAP')
            elif 'performance' in col.lower():
                custom_column_titles[i][col] = col.replace('performance', '\nperformance')
            else:
                # Find a good breaking point
                mid_point = len(words) // 2
                custom_column_titles[i][col] = ' '.join(words[:mid_point]) + '\n' + ' '.join(words[mid_point:])
        else:
            custom_column_titles[i][col] = col

# Define color schemes for each group
color_schemes = ['hdfc_brand', 'corporate', 'gradient_blue', 'vibrant', 'pastel']

# Create complete dashboard
dashboard = create_dashboard(
    df=df,
    sheet_name="Work Status",
    column_groups=column_groups,
    output_path="c:/Code/Ver5-COR-13-HDFC/ceo_dashboard",
    title_prefix="HDFC",
    color_schemes=color_schemes,
    bg_style='presentation',
    custom_titles=custom_titles,
    custom_subtitles=custom_subtitles,
    custom_column_titles=custom_column_titles,
    value_formats=value_formats,
    global_xlabel="Work Status",
    show_plots=True,
    dpi=300
)

## Custom Color Palettes Preview

This section demonstrates the available color palettes that can be used for visualizations.

In [ ]:
# Function to display available color schemes
def display_color_schemes():
    """Display all available color schemes"""
    fig, axes = plt.subplots(len(COLOR_SCHEMES), 1, figsize=(10, len(COLOR_SCHEMES)*1.5))
    
    for i, (name, colors) in enumerate(COLOR_SCHEMES.items()):
        # Create a simple bar chart to showcase the color scheme
        for j, color in enumerate(colors):
            axes[i].bar(j, 1, color=color, edgecolor='white', linewidth=0.5)
        
        axes[i].set_title(f"Color Scheme: {name}")
        axes[i].set_xticks([])
        axes[i].set_yticks([])
        axes[i].set_frame_on(False)
    
    plt.tight_layout()
    plt.suptitle("Available Color Schemes", fontsize=16, y=1.02)
    plt.show()

# Display the color schemes
display_color_schemes()

## Interactive Elements for Presentations (Optional)

This example demonstrates how to add simple interactivity for dynamic presentations.

In [ ]:
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba
import seaborn as sns

# Define a function to highlight a specific category
@widgets.interact(
    category=widgets.Dropdown(options=df.index.tolist(), description='Highlight:'),
    column=widgets.Dropdown(options=df.columns[:4].tolist(), description='Metric:'),
    color=widgets.ColorPicker(description='Color:', value='#FF5733')
)
def highlight_category(category, column, color):
    plt.figure(figsize=(10, 6))
    
    # Create colors with the selected category highlighted
    bar_colors = ['#CCCCCC'] * len(df)
    highlight_idx = df.index.get_loc(category)
    bar_colors[highlight_idx] = color
    
    # Create the bar chart with highlighting
    ax = sns.barplot(x=df.index, y=df[column], palette=bar_colors)
    
    # Add value labels
    for i, p in enumerate(ax.patches):
        ax.annotate(
            f'{p.get_height():.0f}', 
            (p.get_x() + p.get_width() / 2., p.get_height()),
            ha='center', va='bottom', fontweight='bold',
            color='black' if i != highlight_idx else 'white',
            backgroundcolor=None if i != highlight_idx else to_rgba(color, 0.3),
            size=12
        )
    
    plt.title(f"Highlighting {category} in {column}", fontsize=16)
    plt.xlabel("Category")
    plt.ylabel("Value")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## Loading Other Sheets for Analysis

The same visualization approaches can be applied to other sheets in the workbook.

In [ ]:
# Load data from a different sheet
gender_df = pd.read_excel('HDFC_modified.xlsx', index_col="Category", sheet_name="Gender")

# Show the first few rows
gender_df.head()

In [ ]:
# Create a visualization for the Gender sheet
fig, axes = plot_bar_chart(
    df=gender_df,
    columns=gender_df.columns[:4],
    title="Gender Distribution Analysis",
    subtitle="Breakdown by Category and Work Status",
    color_scheme='gradient_red',
    bg_style='presentation',
    bar_edge_color='white',
    bar_width=0.7,
    show_values=True,
    grid=True
)

## Export Dashboard as Images

This section demonstrates how to export the visualizations as high-quality images for presentations.

In [ ]:
# Define a function to export all sheets as dashboards
def export_all_sheet_dashboards(filename, output_dir="c:/Code/Ver5-COR-13-HDFC/exports/", dpi=300):
    """Export dashboards for all sheets in the Excel file"""
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all sheet names
    sheets = pd.ExcelFile(filename).sheet_names
    
    for sheet in sheets:
        print(f"Exporting dashboard for sheet: {sheet}")
        
        # Load the data
        sheet_df = pd.read_excel(filename, index_col="Category", sheet_name=sheet)
        
        # Define column groups - adjust based on number of columns
        n_cols = len(sheet_df.columns)
        column_groups = []
        
        # Group columns into 4-column groups
        for i in range(0, n_cols, 4):
            group = sheet_df.columns[i:i+4].tolist()
            if group:  # Add only non-empty groups
                column_groups.append(group)
        
        # Create dashboard with default options
        dashboard = create_dashboard(
            df=sheet_df,
            sheet_name=sheet,
            column_groups=column_groups,
            output_path=f"{output_dir}HDFC_{sheet}",
            title_prefix="HDFC",
            bg_style='presentation',
            show_plots=False,  # Don't show plots, just save them
            dpi=dpi
        )
        
        print(f"  - Exported {len(dashboard)} chart groups")
    
    print(f"All dashboards exported to {output_dir}")

# Export dashboards for all sheets
# Uncomment to run the export process
# export_all_sheet_dashboards('HDFC_modified.xlsx')

In [ ]:
# Define a function to export all sheets as dashboards
def export_all_sheet_dashboards(filename, output_dir="c:/Code/Ver5-COR-13-HDFC/exports/", dpi=300):
    """Export dashboards for all sheets in the Excel file"""
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get all sheet names
    sheets = pd.ExcelFile(filename).sheet_names
    
    for sheet in sheets:
        print(f"Exporting dashboard for sheet: {sheet}")
        
        # Load the data
        sheet_df = pd.read_excel(filename, index_col="Category", sheet_name=sheet)
        
        # Define column groups - adjust based on number of columns
        n_cols = len(sheet_df.columns)
        column_groups = []
        
        # Group columns into 4-column groups
        for i in range(0, n_cols, 4):
            group = sheet_df.columns[i:i+4].tolist()
            if group:  # Add only non-empty groups
                column_groups.append(group)
        
        # Create dashboard with default options
        dashboard = create_dashboard(
            df=sheet_df,
            sheet_name=sheet,
            column_groups=column_groups,
            output_path=f"{output_dir}HDFC_{sheet}",
            title_prefix="HDFC",
            bg_style='presentation',
            show_plots=False,  # Don't show plots, just save them
            dpi=dpi
        )
        
        print(f"  - Exported {len(dashboard)} chart groups")
    
    print(f"All dashboards exported to {output_dir}")

# Export dashboards for all sheets
# Uncomment to run the export process
# export_all_sheet_dashboards('HDFC_modified.xlsx')